In [1]:
# Set up Notebook
% matplotlib inline

# Standard imports
from sklearn.model_selection import GridSearchCV

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from matplotlib import cm


# We do this to ignore several specific Pandas warnings
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import r2_score

In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
labels = ['clust_logFCS', 'clust_RCSI', 'clust_HDDS']

In [9]:
x_train=train_data.drop(labels, axis=1)
x_train = x_train.drop(x_train.columns[0], axis=1)
x_test=test_data.drop(labels, axis=1)
x_test = x_test.drop(x_test.columns[0], axis=1)

x_train.head()

,clust_L12raincytot,clust_L12day1rain,clust_L12maxdays,clust_floodmax,clust_cells_own,clust_price,clust_thinn,clust_roof,clust_hhsize,clust_hh_age,clust_hh_gender,clust_asset,clust_dist_road,clust_dist_admarc,clust_percent_ag,clust_nutri_reten_constrained,clust_elevation,ipc_lag1,ipc_lag12
0,1090.06130,43,17,0.0,0.1250,3.824215,0.359375,0.1250,5.4375,41.6250,1.2500,-0.327686,1.395625,5.176875,0.6000,0.0000,1311.8750,1.0,NaN
1,855.86176,58,40,0.0,0.4375,3.917409,0.250000,0.1875,5.6875,36.5625,1.3125,-0.202549,0.756875,5.408750,0.6000,0.9375,496.0000,1.0,NaN
2,1300.24370,53,29,0.0,1.5000,3.683867,0.250000,0.5000,6.3750,40.4375,1.3750,0.548275,0.163125,17.038126,0.4500,1.0000,526.7500,1.0,NaN
3,1036.97120,52,29,0.0,0.6250,4.061391,0.500000,0.2500,5.4375,46.8750,1.1875,-0.077412,5.803750,7.724375,0.4375,0.0000,564.1875,1.0,NaN
4,952.18280,54,25,0.0,0.5625,3.879500,0.250000,0.0625,7.0000,43.8125,1.0000,-0.202549,11.286875,11.103125,0.4875,1.0000,1539.9375,1.0,NaN


In [10]:
y_train = train_data[labels]
y_test = test_data[labels]
y_test.head()

,clust_logFCS,clust_RCSI,clust_HDDS
0,3.666015,12.312500,4.687500
1,3.718879,5.352941,5.235294
2,3.799830,5.080000,5.720000
3,3.844891,4.285714,5.095238
4,3.687936,0.812500,5.312500


In [11]:
from sklearn.preprocessing import Imputer,StandardScaler

imp = Imputer(missing_values='NaN', strategy='mean', axis=0, copy=False)
x_train = imp.fit_transform(x_train)
x_test = imp.fit_transform(x_test)

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)


In [7]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)
x_train = poly.fit_transform(x_train)
x_test =  poly.fit_transform(x_test)



In [20]:
import numpy as np
y_train_new = np.arcsinh(np.array ( y_train[labels[1]])) 
y_test_new = np.arcsinh(np.array (y_test[labels[1]])) 

In [21]:
from sklearn.linear_model import LinearRegression

# Create and fit our linear regression model to training data
model = LinearRegression(fit_intercept=True)
model.fit(x_train, y_train_new)

# Compute model predictions for test data
pred = model.predict(x_test)

actual =y_test_new
r2_linear = stats.pearsonr(actual, pred)[0] ** 2
r2_linear

0.12297431987562661

In [22]:
# fit on logFCS 
from sklearn.ensemble import RandomForestRegressor

# Create Regressor with default properties
rfc = RandomForestRegressor(random_state =0,n_jobs =4,warm_start = True)

parameters = {'max_depth':np.arange( 1,4, 1 ).tolist(), 'min_samples_leaf':np.arange( 1, 4, 1 ).tolist()}
clf = GridSearchCV(rfc, parameters,cv=6, n_jobs= 4, iid = True,  refit= True,pre_dispatch= '2*n_jobs')
clf.fit(x_train, y_train_new)

# Fit estimator and display score

# Regress on test data
pred = clf.predict(x_test)

actual =y_test_new
r2_rfc = stats.pearsonr(actual, pred)[0] ** 2
r2_rfc

0.04498710944727587

In [147]:
# fit on logFCS 
from sklearn.ensemble import RandomForestRegressor

# Create Regressor with default properties
rfc = RandomForestRegressor(random_state =0,n_jobs =4,warm_start = True,max_depth=4, min_samples_leaf=3 )

rfc.fit(x_train, y_train_new)
# Fit estimator and display score

# Regress on test data
pred = rfc.predict(x_test)

actual =y_test_new
r2_rfc = stats.pearsonr(actual, pred)[0] ** 2
r2_rfc

0.04181011537703592

In [34]:
from sklearn.linear_model import RidgeCV
ridge = RidgeCV(alphas=(0.2,0.6,0.01), fit_intercept=True, normalize=False, scoring=None, cv=5, gcv_mode='auto', store_cv_values=False) 

# Define different alpha values for different fits
# alpha = [0.0, 1E-6, 1E-4, 1E-2, 1.0]

ridge.fit(x_train, y_train_new)
pred = ridge.predict(x_test)

actual =y_test_new
r2_ridge= stats.pearsonr(actual, pred)[0] ** 2
r2_ridge

0.12296352897584324

In [184]:
from sklearn.linear_model import BayesianRidge
bridge = BayesianRidge(compute_score=True)
bridge.fit(x_train, y_train_new)
pred = bridge.predict(x_test)

actual =y_test_new
r2_bridge= stats.pearsonr(actual, pred)[0] ** 2
r2_bridge

0.12311634209636622

In [142]:
from sklearn.linear_model import LassoCV

ls = LassoCV(eps=0.001, n_alphas=100, alphas=(2.1,3,0.1), fit_intercept=True,precompute='auto',n_jobs=4, random_state=0, selection='cyclic')

ls = ls.fit(x_train, y_train_new)
pred = ls.predict(x_test)

actual =y_test_new
r2_ls= stats.pearsonr(actual, pred)[0] ** 2
r2_ls

0.12092485754124149

In [143]:
from sklearn.linear_model import ElasticNetCV
en = ElasticNetCV(alphas=(1,4,0.1), copy_X=True, cv=5, eps=0.01, fit_intercept=True,
       l1_ratio=0.3, max_iter=1000, n_alphas=100, n_jobs=1,
       normalize=False, positive=False, precompute='auto', random_state=0,
       selection='cyclic', tol=0.0001, verbose=0)

en.fit(x_train, y_train_new)

pred = en.predict(x_test)

actual =y_test_new
r2_en= stats.pearsonr(actual, pred)[0] ** 2
r2_en


0.12006059193359798

In [144]:
from sklearn.ensemble import GradientBoostingRegressor

# Create Regressor with default properties
gbr = GradientBoostingRegressor(random_state=0,learning_rate=0.4, n_estimators=9,subsample=1, criterion='friedman_mse', min_samples_split=2)

gbr.fit(x_train, y_train_new)
pred = gbr.predict(x_test)

actual =y_test_new
r2_gbr= stats.pearsonr(actual, pred)[0] ** 2
r2_gbr


0.06581221706443534

In [185]:
df = pd.DataFrame(actual)
df["pred"] = pred
df["actual"]=actual
df.to_csv('rcsi.csv')

In [183]:
import seaborn as sns

# Create a linear model plot
sns.lmplot(pred, actual,df);

KeyError: '[0.09509471 0.14944313 0.26021146 0.36238645 0.59014369 0.6112243\n 0.66433062 0.69314718 0.72442749 0.74239938 0.78388429 0.80058527\n 0.81560892 0.9307718  0.97066545 0.97458795 0.98873638 1.01166966\n 1.03124095 1.04759301 1.06971075 1.09144892 1.09371574 1.14635208\n 1.15107199 1.17944404 1.18532247 1.20361645 1.2092725  1.22939201\n 1.23752966 1.25167473 1.25175675 1.25771931 1.27102712 1.27709082\n 1.27710412 1.28379563 1.29278711 1.31203311 1.3163993  1.31771843\n 1.33178905 1.33959751 1.34196347 1.34517913 1.36651524 1.38140346\n 1.38612459 1.40405705 1.40563117 1.40898812 1.41533008 1.43408623\n 1.44363548 1.44936111 1.45358378 1.45983148 1.46820817 1.47521817\n 1.48089992 1.48186076 1.48276017 1.48592073 1.50092044 1.50300739\n 1.50575677 1.50805916 1.50808078 1.50877332 1.52056927 1.52593129\n 1.52801629 1.52850819 1.53602125 1.53881785 1.54368973 1.54786547\n 1.55015796 1.55425274 1.5545636  1.55567537 1.55934966 1.56052152\n 1.56556828 1.56958279 1.57027854 1.57

In [186]:
df

,0,pred,actual
0,3.205407,2.134089,3.205407
1,2.379406,1.534749,2.379406
2,2.328008,1.413559,2.328008
3,2.161776,1.960183,2.161776
4,0.742399,2.094010,0.742399
5,3.078704,1.585790,3.078704
6,1.481861,1.699915,1.481861
7,1.508773,1.946352,1.508773
8,1.846809,1.766949,1.846809
9,1.550158,2.616829,1.550158
